# Sistema de recomendaciones basado en Contenido
## Universidad del Valle de Guatemala <br> Facultad de Ingeniería
#### Departamento de Ciencias de la Computación <br> Deep Learning y Sistemas Inteligentes - Sección 20 <br><br> Jeyner Arango 201106 <br> Cristian Laynez 201281


In [1]:
# Importar las librerias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from keras.models import Model
from keras.layers import Input, Embedding, Concatenate, Flatten, Dense

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
books_data = pd.read_csv("./data/_best_scores.csv")
books_data.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Location,Age
0,105221,0590353403,10,Harry Potter and the Sorcerer's Stone (Book 1),J. K. Rowling,"la mesa, california, usa",15.0
1,105214,1573220221,10,The Color of Water: A Black Man's Tribute to H...,James McBride,"pittsburgh, pennsylvania, usa",34.0
2,26421,1558743669,10,"A Child Called \It\"": One Child's Courage to S...",Dave Pelzer,"st. albans, west virginia, usa",-1.0
3,105221,0064407667,10,The Bad Beginning (A Series of Unfortunate Eve...,Lemony Snicket,"la mesa, california, usa",15.0
4,105221,0064407675,10,The Reptile Room (A Series of Unfortunate Even...,Lemony Snicket,"la mesa, california, usa",15.0


### Ahora vamos a construir un modelo para crear el sistema de recomendación

In [3]:
len(books_data)

25000

In [4]:
books_data["Book-Title"].head()

0       Harry Potter and the Sorcerer's Stone (Book 1)
1    The Color of Water: A Black Man's Tribute to H...
2    A Child Called \It\": One Child's Courage to S...
3    The Bad Beginning (A Series of Unfortunate Eve...
4    The Reptile Room (A Series of Unfortunate Even...
Name: Book-Title, dtype: object

In [5]:
tfidf = TfidfVectorizer(stop_words="english")

books_data["Book-Title"] = books_data["Book-Title"].fillna(" ")

tfidf_matrix = tfidf.fit_transform(books_data["Book-Title"])

tfidf_matrix.shape

(25000, 15654)

In [6]:
tfidf_matrix = tfidf_matrix.astype(np.float32)

tfidf_matrix = tfidf_matrix[:25000, :25000]

tfidf_matrix = tfidf_matrix.toarray()

cosine_sim_books = cosine_similarity(tfidf_matrix, tfidf_matrix)

cosine_sim_books.shape

(25000, 25000)

In [7]:
def get_book_recommendations(book_title, cosine_sim_matrix, df, top_n=10):
    indices = pd.Series(df.index, index=df["Book-Title"]).drop_duplicates()
    book_index = indices[book_title]

    similarity_scores = pd.DataFrame(cosine_sim_matrix[book_index], columns=["score"])
    book_indices = similarity_scores.sort_values("score", ascending=False)[1:top_n+1].index

    recommended_books = df["Book-Title"].iloc[book_indices]
    return recommended_books

In [8]:
target_book = "Mr. Wroe's Virgins"
book_recommendations = get_book_recommendations(target_book, cosine_sim_books, books_data)
print(f"Book recommendations for '{target_book}':")
print(book_recommendations)

Book recommendations for 'Mr. Wroe's Virgins':
4965     Virgins of Paradise
15019                  Mr. X
3739              Mr. Murder
22469             Mr. Murder
1303              Mr. Murder
16757            Mr. Perfect
5487             Mr. Perfect
4005             Mr. Perfect
690              Mr. Perfect
10644            Mr. Perfect
Name: Book-Title, dtype: object
